<a href="https://colab.research.google.com/github/gabrielnichio/book-recomendation-NLP/blob/main/book_recomendation_dt_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import torch
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [58]:
data = pd.read_csv("/content/data.csv")
data.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [59]:
books = data[["title", "authors", "categories", "thumbnail", "description", "average_rating"]].dropna()
books.head()

,title,authors,categories,thumbnail,description,average_rating
0,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,3.85
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,3.83
2,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,3.97
3,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",3.93
4,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,4.15


In [60]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6215 entries, 0 to 6809
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           6215 non-null   object 
 1   authors         6215 non-null   object 
 2   categories      6215 non-null   object 
 3   thumbnail       6215 non-null   object 
 4   description     6215 non-null   object 
 5   average_rating  6215 non-null   float64
dtypes: float64(1), object(5)
memory usage: 339.9+ KB


In [61]:
def first_treatement(text):
  text = re.sub(r'\W', ' ', text.lower())
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words("english"))

  return ' '.join([word for word in tokens if word not in stop_words])

books['treatment_1'] = books['description'].apply(first_treatement)
books.head()

,title,authors,categories,thumbnail,description,average_rating,treatment_1
0,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,3.85,novel readers critics eagerly anticipating dec...
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,3.83,new christie christmas full length novel adapt...
2,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,3.97,volume two stephen donaldson acclaimed second ...
3,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",3.93,memorable mesmerizing heroine jennifer brillia...
4,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,4.15,lewis work nature love divides love four categ...


In [62]:
books = books[books['treatment_1'] != '']
books.drop_duplicates(subset=['treatment_1'], inplace=True)
books.head()

,title,authors,categories,thumbnail,description,average_rating,treatment_1
0,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,3.85,novel readers critics eagerly anticipating dec...
1,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,3.83,new christie christmas full length novel adapt...
2,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,3.97,volume two stephen donaldson acclaimed second ...
3,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",3.93,memorable mesmerizing heroine jennifer brillia...
4,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,4.15,lewis work nature love divides love four categ...


In [63]:
books.size

42994

In [64]:
main_categories = books["categories"].value_counts().loc[lambda x: x > 30].index.tolist()

books = books[books["categories"].isin(main_categories)]

books.size

36148

In [72]:
books.to_csv('books.csv')

# Model testing

In [65]:
sentence = "I'm looking for a Renaissance art book that dives into the techniques of masters like Leonardo da Vinci and Michelangelo, with detailed analyses of their iconic works and the cultural impact of their time."

In [66]:
model_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

model = hub.load(model_url)

In [67]:
descriptions = books['treatment_1'].to_list()
descriptions_emb = model(descriptions)

In [68]:
cosine_similarity(descriptions_emb)

array([[0.9999999 , 0.37997293, 0.41597325, ..., 0.41883576, 0.27387258,
        0.26622304],
       [0.37997293, 1.        , 0.43594998, ..., 0.26449525, 0.04086119,
        0.24815895],
       [0.41597325, 0.43594998, 1.0000001 , ..., 0.33511144, 0.12487306,
        0.17221805],
       ...,
       [0.41883576, 0.26449525, 0.33511144, ..., 1.        , 0.48568574,
        0.19721913],
       [0.27387258, 0.04086119, 0.12487306, ..., 0.48568574, 1.0000002 ,
        0.15036911],
       [0.26622304, 0.24815895, 0.17221805, ..., 0.19721913, 0.15036911,
        1.0000001 ]], dtype=float32)

In [69]:
def sentences_similarities(sentence, descriptions, descriptions_emb, top_n=5):
  sentence_emb = model([sentence])
  similarities = cosine_similarity(sentence_emb, descriptions_emb).flatten()

  top_books = np.argsort(-similarities)

  for idx in top_books[:top_n]:
    book = books.iloc[idx]

    print('Book: ', book['title'])
    print('Author(s): ', book['title'])
    print('Description: ', book['description'])
    print('Category: ', book['categories'])
    print('Average rating: ', book['average_rating'])
    print('\n')


In [70]:
sentences_similarities(sentence, descriptions, descriptions_emb)

Book:  The Agony and the Ecstasy
Author(s):  The Agony and the Ecstasy
Description:  A classic historical novel honoring the five hundredth anniversary of the artist's famed David sculpture dramatizes the life of the Renaissance artistic genius Michelangelo, recalls his love affairs, his disputes with cardinals and popes, and his years of working on the Sistine Chapel. Reprint.
Category:  Fiction
Average rating:  4.07


Book:  Discovering Great Artists
Author(s):  Discovering Great Artists
Description:  Featuring more than 150 activities, this guide teaches the styles, works, and techniques of the great masters—Van Gogh, Michelangelo, Rembrandt, and more.
Category:  Juvenile Nonfiction
Average rating:  4.24


Book:  The Notebooks of Leonardo Da Vinci
Author(s):  The Notebooks of Leonardo Da Vinci
Description:  Throughout his life Leonardo da Vinci carried notebooks in which he scribbled down ideas and opinions as they occurred - personal, domestic, scientific, philosophical, artistic -